<a href="https://colab.research.google.com/github/GoetheUniCV/cvproject/blob/main/Luis/covid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Grad-CAM for COVID-19

##1. Setup
###1.1 Python Environment
Almost all required packages are preinstalled, except _kerastuner_. You can install it with `!pip install -U keras-tuner`.

###1.2 Google Drive
####1.2.1 Structure
Create the following directory structure in your Drive:
```
MyDrive
└── cvproject
    ├── COVID-10 Radiography Database
    │   ├── COVID-19
    │   ├── NORMAL
    │   └── Viral Pneumonia
    └── models
```
####1.2.2 Upload COVID-19 Dataset
The directory _COVID-10 Radiography Database_ must contain the dataset from Kaggle (https://www.kaggle.com/tawsifurrahman/covid19-radiography-database).

___


In [1]:
!pip install -U keras-tuner

Requirement already up-to-date: keras-tuner in /usr/local/lib/python3.6/dist-packages (1.0.2)


In [3]:
import os
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
import kerastuner as kt
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameter as hp
from keras.layers import Dense,Dropout,Activation,Add,MaxPooling2D,Conv2D,Flatten
from keras.models import Sequential 
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG19
from keras import layers
from keras.preprocessing import image
from google.colab import drive

In [5]:
# This mounts your Google Drive to the Colab VM.
drive.mount('/content/drive', force_remount=True)

# Foldername in your Drive where you have saved the COVID-19 data.
PROJECT_PATH = '/content/drive/My Drive/cvproject/'
DATABASE_PATH = f'{PROJECT_PATH}/COVID-19 Radiography Database'
MODELS_PATH = f'{PROJECT_PATH}/models'

#assert FOLDERNAME is not None, "[!] Enter the foldername."

# now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
#sys.path.append(FOLDERPATH)

Mounted at /content/drive


In [7]:
# Check if database is complete.
try:
  if len(os.listdir(f'{DATABASE_PATH}/COVID-19')) == 1143:
    print('COVID-19 compelte.')
  else:
    print('COVID-19 incompelte.')

  if len(os.listdir(f'{DATABASE_PATH}/NORMAL')) == 1341:
    print('NORMAL compelte.')
  else:
    print('NORMAL incompelte.')

  if len(os.listdir(f'{DATABASE_PATH}/Viral Pneumonia')) == 1345:
    print('Viral Pneumonia compelte.')
  else:
    print('Viral Pneumonia incompelte.')
except FileNotFoundError:
  print("Warning: Some directories are missing!")



COVID-19 compelte.
NORMAL compelte.
Viral Pneumonia compelte.


In [ ]:
train_data = defect_tree = tf.keras.preprocessing.image_dataset_from_directory(
    '.',
    labels="inferred",
    label_mode="int",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(244, 244),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset="training",
    interpolation="bilinear",
    follow_links=False,
)
test_data = tf.keras.preprocessing.image_dataset_from_directory(
    '.',
    labels="inferred",
    label_mode="int",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(244, 244),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset="validation",
    interpolation="bilinear",
    follow_links=False,
)

Found 3829 files belonging to 3 classes.
Using 3064 files for training.
Found 3829 files belonging to 3 classes.
Using 765 files for validation.


In [ ]:
#classes 
class_names =  ["COVID-19","NORMAL","Viral Pneumonia"]
for i in range(len(class_names)):
    print(class_names[i] ," " , i)

COVID-19   0
NORMAL   1
Viral Pneumonia   2


In [ ]:
# Defining the CNN.
model = tf.keras.models.Sequential([
  layers.BatchNormalization(),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.3),
  layers.Conv2D(128, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(256, activation='relu'),
  layers.Dropout(0.15),
  layers.Dense(3, activation= 'softmax')
])

In [ ]:
# Avoid overfitting...
early = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5)

In [ ]:
# Compile the model.
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Fit the model.
history=model.fit(train_data,validation_data= test_data,batch_size=32,epochs = 20,callbacks=[early])

Epoch 1/20
96/96 [==============================] - 1692s 16s/step - loss: 4.2358 - accuracy: 0.6253 - val_loss: 0.5082 - val_accuracy: 0.8157
Epoch 2/20
96/96 [==============================] - 55s 536ms/step - loss: 0.2477 - accuracy: 0.9167 - val_loss: 0.2702 - val_accuracy: 0.9111
Epoch 3/20
96/96 [==============================] - 54s 533ms/step - loss: 0.1594 - accuracy: 0.9408 - val_loss: 0.2325 - val_accuracy: 0.9242
Epoch 4/20
96/96 [==============================] - 54s 533ms/step - loss: 0.1292 - accuracy: 0.9515 - val_loss: 0.1998 - val_accuracy: 0.9386
Epoch 5/20
96/96 [==============================] - 54s 532ms/step - loss: 0.0890 - accuracy: 0.9681 - val_loss: 0.3876 - val_accuracy: 0.9072
Epoch 6/20
96/96 [==============================] - 55s 537ms/step - loss: 0.0898 - accuracy: 0.9668 - val_loss: 0.2241 - val_accuracy: 0.9307
Epoch 7/20
96/96 [==============================] - 55s 538ms/step - loss: 0.0476 - accuracy: 0.9786 - val_loss: 0.1957 - val_accuracy: 0.9490

In [ ]:
# Evaluate the model.
model.evaluate(test_data)

24/24 [==============================] - 10s 297ms/step - loss: 0.2487 - accuracy: 0.9477


[0.24871842563152313, 0.9477124214172363]

In [ ]:
#plotting training values
sns.set()

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)

#accuracy plot
plt.plot(epochs, acc, color='green', label='Training Accuracy')
plt.plot(epochs, val_acc, color='blue', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()

plt.figure()
#loss plot
plt.plot(epochs, loss, color='pink', label='Training Loss')
plt.plot(epochs, val_loss, color='red', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
#train data accuracy and loss
plt.plot(history.history['val_loss'], label = 'training loss')
plt.plot(history.history['val_accuracy'], label = 'training accuracy')
plt.legend()

In [ ]:
#test data accuracy and loss
plt.plot(history.history['loss'], label = 'training loss')
plt.plot(history.history['accuracy'], label = 'training accuracy')
plt.legend()

In [ ]:
#save file as h5 model
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5

model.save(f'{MODELS_PATH}/model.h5')

In [16]:
model = keras.models.load_model(f'{MODELS_PATH}/model.h5')

In [17]:
model